In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import *
%matplotlib notebook
from sklearn.linear_model import LogisticRegression

In [ ]:
RP884 = pd.read_csv('db_measurements_v2.1.0.csv') # ASHRAE RP-884 thermal comfort dataset

In [ ]:
"""
your inputs: air temperature, radiant temperature, relative humidity,
             air velocity, metabolic rate, clothing level
your output: thermal sensation
"""
Data = RP884[['thermal_sensation','ta', 'tr', 'rh', 'vel', 'met', 'clo']].dropna()

y = Data['thermal_sensation'].to_numpy()
y[y<-0.5]=-1
y[(y>=-0.5)&(y<=0.5)]=0
y[y>0.5]=1

x = Data[['ta', 'tr', 'rh', 'vel', 'met', 'clo']].to_numpy()

In [ ]:
plt.bar(-1,(y==-1).sum(),1,color='b')
plt.bar(0,(y==0).sum(),1,color='g')
plt.bar(1,(y==1).sum(),1,color='r')
plt.title('Thermal Sensation')
plt.xticks([-1,0,1], ['cold', 'neutral', 'hot'])
plt.show()

In [ ]:
fig,ax = plt.subplots(2,3, figsize=(15,8))
ax[0,0].hist(x[:,0])
ax[0,1].hist(x[:,1])
ax[0,2].hist(x[:,2])
ax[1,0].hist(x[:,3])
ax[1,1].hist(x[:,4])
ax[1,2].hist(x[:,5])
ax[0,0].title.set_text('Air Temperature')
ax[0,1].title.set_text('Radiant Temperature')
ax[0,2].title.set_text('Relative Humidity')
ax[1,0].title.set_text('Air Velocity')
ax[1,1].title.set_text('Metabolic Rate')
ax[1,2].title.set_text('Clothing level')

In [ ]:
model=LogisticRegression(solver='lbfgs', max_iter=1000).fit(x,y)

In [ ]:
ta_plot = np.linspace(10,35,100)

def get_predict_proba(rh=50, vel=0.1, met=1.1, clo=0.6):
    tr_plot = ta_plot
    rh_plot = np.ones_like(ta_plot)*rh
    vel_plot = np.ones_like(ta_plot)*vel
    met_plot = np.ones_like(ta_plot)*met
    clo_plot = np.ones_like(ta_plot)*clo
    x_plot = np.vstack((ta_plot,tr_plot,rh_plot,vel_plot,met_plot,clo_plot)).T

    return model.predict_proba(x_plot)

predict_proba = get_predict_proba(rh=50, vel=0.1, met=1.1, clo=0.6)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
line1, = ax.plot(ta_plot, predict_proba[:,0], color='b')
line2, = ax.plot(ta_plot, predict_proba[:,1], color='g')
line3, = ax.plot(ta_plot, predict_proba[:,2], color='r')
ax.set_ylim(0,1)
ax.set_xlabel('Air and Radiant Temperatures')
ax.set_ylabel('Probability')

def update(rh=50, vel=0.1, met=1.1, clo=0.6):
    predict_proba = get_predict_proba(rh, vel, met, clo)

    line1.set_ydata(predict_proba[:,0])
    line2.set_ydata(predict_proba[:,1])
    line3.set_ydata(predict_proba[:,2])
    
    fig.canvas.draw_idle()

interact(update,
         rh=(0,100,10),
         vel=(0,4,0.1),
         met=(0.7,4,0.1),
         clo=(0,4,0.1));